In [1]:
# Import the necessary libraries
%pip install autogluon
import numpy as np
import pandas as pd

from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import KFold
import pickle

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score as acs_score

from src.data.data_fetcher import get_raw_data
from src.features.preprocess_data import get_preprocessed_test_data, fetch_preprocessed_data
pd.set_option('display.max_columns', 200)

from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = get_raw_data()

X_train_obs_combined, X_val_obs_combined, y_train_obs_combined, y_val_obs_combined, X_train_est_combined, X_val_est_combined, y_train_est_combined, y_val_est_combined = fetch_preprocessed_data()
x_test_whole = get_preprocessed_test_data()

x_whole = pd.concat([X_train_obs_combined, X_val_obs_combined, X_train_est_combined, X_val_est_combined])
y_whole = pd.concat([y_train_obs_combined, y_val_obs_combined, y_train_est_combined, y_val_est_combined])
x_whole.reset_index(drop=True, inplace=True)
y_whole.reset_index(drop=True, inplace=True)


x_whole_obs = pd.concat([X_train_obs_combined, X_val_obs_combined])
y_whole_obs = pd.concat([y_train_obs_combined, y_val_obs_combined])

x_whole_est = pd.concat([X_train_est_combined, X_val_est_combined])
y_whole_est = pd.concat([y_train_est_combined, y_val_est_combined])

x_whole_est.head(10)

In [ ]:
x_whole["pv_measurement"] = y_whole
df_shuffled = x_whole.sample(frac=1, random_state=42).reset_index(drop=True)
x_whole_a = df_shuffled[df_shuffled['location_a'] == 1]
x_whole_b = df_shuffled[df_shuffled['location_b'] == 1]
x_whole_c = df_shuffled[df_shuffled['location_c'] == 1]

y_whole_a = x_whole_a["pv_measurement"]
x_whole_a = x_whole_a.drop('location_a', axis = 1)
x_whole_a = x_whole_a.drop('location_b', axis = 1)
x_whole_a = x_whole_a.drop('location_c', axis = 1)

y_whole_b = x_whole_b["pv_measurement"]
x_whole_b = x_whole_b.drop('location_a', axis = 1)
x_whole_b = x_whole_b.drop('location_b', axis = 1)
x_whole_b = x_whole_b.drop('location_c', axis = 1)

y_whole_c = x_whole_c["pv_measurement"]
x_whole_c = x_whole_c.drop('location_a', axis = 1)
x_whole_c = x_whole_c.drop('location_b', axis = 1)
x_whole_c = x_whole_c.drop('location_c', axis = 1)
cat_features = ["time_since_prediction"]

In [ ]:
from autogluon.tabular import TabularPredictor

# Define the hyperparameters for CatBoost
catboost_params = {
    'max_depth': 9,
    'cat_features': ['time_since_prediction'],
    'loss_function': 'MAE',
    'verbose': 100
}

# Prepare the hyperparameters for AutoGluon
hyperparameters = {
    'CAT': catboost_params,
    'XGB': {},  # Default parameters for XGBoost
    'GBM': {},  # Default parameters for LightGBM
}

# Initialize the TabularPredictor
best_model_a = TabularPredictor(label='pv_measurement').fit(
    train_data=x_whole_a,
    presets='best_quality',
    hyperparameters=hyperparameters
)


In [ ]:
from autogluon.tabular import TabularPredictor

# Define the hyperparameters for CatBoost
catboost_params = {
    'max_depth': 9,
    'cat_features': ['time_since_prediction'],
    'loss_function': 'MAE',
    'verbose': 100
}

# Prepare the hyperparameters for AutoGluon
hyperparameters = {
    'CAT': catboost_params,
    'XGB': {},  # Default parameters for XGBoost
    'GBM': {},  # Default parameters for LightGBM
}

# Initialize the TabularPredictor
best_model_b = TabularPredictor(label='pv_measurement').fit(
    train_data=x_whole_b,
    presets='best_quality',
    hyperparameters=hyperparameters
)


In [ ]:
from autogluon.tabular import TabularPredictor

# Define the hyperparameters for CatBoost
catboost_params = {
    'max_depth': 9,
    'cat_features': ['time_since_prediction'],
    'loss_function': 'MAE',
    'verbose': 100
}

# Prepare the hyperparameters for AutoGluon
hyperparameters = {
    'CAT': catboost_params,
    'XGB': {},  # Default parameters for XGBoost
    'GBM': {},  # Default parameters for LightGBM
}

# Initialize the TabularPredictor
best_model_c = TabularPredictor(label='pv_measurement').fit(
    train_data=x_whole_c,
    presets='best_quality',
    hyperparameters=hyperparameters
)


In [ ]:
x_whole_a = x_test_whole[x_test_whole['location_a'] == 1]
x_whole_b = x_test_whole[x_test_whole['location_b'] == 1]
x_whole_c = x_test_whole[x_test_whole['location_c'] == 1]

x_whole_a = x_whole_a.drop('location_a', axis = 1)
x_whole_a = x_whole_a.drop('location_b', axis = 1)
x_whole_a = x_whole_a.drop('location_c', axis = 1)

x_whole_b = x_whole_b.drop('location_a', axis = 1)
x_whole_b = x_whole_b.drop('location_b', axis = 1)
x_whole_b = x_whole_b.drop('location_c', axis = 1)

x_whole_c = x_whole_c.drop('location_a', axis = 1)
x_whole_c = x_whole_c.drop('location_b', axis = 1)
x_whole_c = x_whole_c.drop('location_c', axis = 1)

In [ ]:
y_pred_a = best_model_a.predict(x_whole_a)
y_pred_b = best_model_b.predict(x_whole_b)
y_pred_c = best_model_c.predict(x_whole_c)
y_pred = pd.concat([pd.Series(y_pred_a), pd.Series(y_pred_b), pd.Series(y_pred_c)])
y_pred = y_pred.reset_index(drop=True)
# Save the model
from src.models.saving import save_predictions
from src.features.postprocess_data import postprocess_data

y_pred = postprocess_data(x_test_whole, pd.DataFrame(y_pred))
save_predictions(y_pred, 'autogluon gunnar')